Признаки:
1) сколько дат из будущего времени; 2) сколько интервалов длиной больше недели (двух недель/месяца?).

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
import xgboost

from numpy import loadtxt
from xgboost import XGBClassifier
import time
import pickle

In [4]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
import gensim
import pymorphy2
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn import preprocessing

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier

In [5]:
import tqdm

In [6]:
import re

In [7]:
def read_txt(path,class_of_t):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            f = open(open_name)
            try:
                cleantext = BeautifulSoup(f, 'lxml').text
                yield class_of_t, cleantext, filename
#                 yield open_name
                f.close()
            except:
                f.close()
                pass

In [8]:
data = []

In [9]:
for i in read_txt('/Users/Valeriya/Desktop/Interfax/Анонс_нерасп','Анонс'):
    data.append(i)
#     with open('anonce.txt', 'a') as f:
#         f.write(re.findall('[0-9]_(.*)', i)[0].strip('.html'))
#         f.write('\n')

In [10]:
for i in read_txt('/Users/Valeriya/Desktop/Interfax/Анонс_расп', 'Анонс'):
    data.append(i)
    
# #     with open('anonce.txt', 'a') as f:
# #         f.write(re.findall('[0-9]_(.*)', i)[0].strip('.html'))
# #         f.write('\n')

In [11]:
for i in read_txt('/Users/Valeriya/Desktop/Interfax/Остальные_жанры', 'Остальное'):
    data.append(i)
    
#     with open('other.txt', 'a') as f:
#         f.write(re.findall('[0-9]_(.*)', i)[0].strip('.html'))
#         f.write('\n')

In [12]:
for i in tqdm.tqdm_notebook(read_txt('/Users/Valeriya/Desktop/Interfax/15000_other_genres', 'Остальное')):
    data.append(i)

In [13]:
# len(data)

In [14]:
df = pd.DataFrame(data, columns=['y', 'X', 'filename'])

In [15]:
len(df.filename)

15401

In [16]:
num = 0

In [17]:
features_arr = []

In [18]:
p = '/Users/Valeriya/Desktop/Interfax/all_features/features_15000_other_genres/dateFeature_eastrussianewswrm_219131372.txt'

In [46]:
tr = '/Users/Valeriya/Desktop/Interfax/all_features/features_15000_other_genres/dateFeature_echochelnewswrm_218481001.txt'

In [22]:
f = open(p).readlines()

In [219]:
# n_dates_fut = 0
# n_dates_2weeks = 0
# for row in f:
#     nums = row.split('|')[0].split('\t')
#     num_fut = nums[0].strip('\ufeff').strip()
#     num_interval = nums[1].strip('\ufeff').strip()
#     n_dates_fut +=1
#     if int(num[1]) >= 14:
#         n_dates_2weeks += 1


In [18]:
n_dates_fut_array = []
n_dates_interval = []
path1 = '/Users/Valeriya/Desktop/Interfax/all_features/features_15000_other_genres/'
path2 = '/Users/Valeriya/Desktop/Interfax/features_all_22november/'
curr_path = ''
for i in tqdm.tqdm_notebook(df.filename):
    if i.endswith('html'):
        fromdf = i.strip('.html')
        fromdf = re.sub('[0-9]+_', '', fromdf)
    elif i.endswith('xml'):
        fromdf = i.strip('.xml')
        fromdf = re.sub('[0-9]+_', '', fromdf)
        
    name = 'dateFeature_'+fromdf+'.txt'
    
    if os.path.exists(path2 + name):
        curr_path = path2
    elif os.path.exists(path1+name):
        curr_path = path1
    else:
        n_dates_fut_array.append(None)
        n_dates_interval.append(None)
        continue
        
#         print('ok')
    f = open(curr_path+name)
    file = f.readlines()
#     feature_vec_for_row = []
    n_dates_fut = 0
    n_dates_2weeks = 0
    for row in file:

        nums = row.split('|')[0].split('\t')
        if len(nums) == 2:
            num_fut = nums[0].strip('\ufeff').strip()
            num_interval = nums[1].strip('\ufeff').strip()
            n_dates_fut +=1
            if int(num_interval) >= 14:
                n_dates_2weeks += 1
        elif len(nums) == 1:
            pass
    
    n_dates_fut_array.append([n_dates_fut])
    n_dates_interval.append([n_dates_2weeks])


#     features_arr.append(feature_vec_for_row)

            
            
#             f.close()
#             features_arr.append(nums)

In [221]:
# num_ann = 0
# for item in features_arr:
#     if item:
#         if item[1] <1 and item[0] > 1:
#             num_ann+=1

In [222]:
# len(n_dates_interval)

In [19]:
len(n_dates_fut_array)

15401

In [20]:
# feat

In [21]:
df['n_of_dates_fut'] = n_dates_fut_array
df['n_of_intervals'] = n_dates_interval

In [231]:
# del df['new_features']

In [97]:
# formal_annonce = df[df.y == 'Анонс'].form_feat
# formal_other = df[df.y == 'Остальное'].form_feat

In [232]:
# del df['form_feat']

In [21]:
# formal_annonce

In [23]:
# # 1st number from rows
# fst_nums_annonce = []
# for i in formal_annonce:
#     try:
#         for arr in i:
#             if arr[0] is not None:
#                 if arr[0] == 833579:
#                     print('!')
#                 fst_nums_annonce.append(arr[0])
            
#     except:
#         pass

In [24]:
# fst_nums_other = []
# for i in enumerate(formal_other):
#     try:
#         for arr in i[1]:
#             if arr[0] is not None:
#                 if arr[0] == 833579:
#                     print(i[0])
#                 fst_nums_other.append(arr[0])
            
#     except:
#         pass

In [76]:
# fst_nums_annonce = []

In [23]:
# for index, row in df[df.y == 'Анонс'].iterrows():
#     ok_row = row['form_feat']
#     if ok_row is not None:
#         for arr in ok_row:
#             try:
#                 if arr[0] is not None:
#                     fst_nums_annonce.append(arr[0])
#             except:
#                 pass

In [24]:
# fst_nums_annonce

In [77]:
# fst_nums_other = []
# for index, row in df[df.y == 'Остальное'].iterrows():
#     ok_row = row['form_feat']
#     if ok_row is not None:
#         for arr in ok_row:
#             try:
#                 if arr[0] is not None:
#                     if arr[0] == 833579:
#                         print(index)
#                     fst_nums_other.append(arr[0])
#             except:
#                 pass

In [22]:
import numpy as np

In [27]:
# df.iloc[2412]

In [79]:
# np.mean(fst_nums_annonce), np.median(fst_nums_annonce), np.max(fst_nums_annonce), np.min(fst_nums_annonce)

In [80]:
# np.mean(fst_nums_other), np.median(fst_nums_other), np.max(fst_nums_other), np.min(fst_nums_other)

In [81]:
# last_nums_annonce = []
# for index, row in df[df.y == 'Анонс'].iterrows():
#     ok_row = row['form_feat']
#     if ok_row is not None:
#         for arr in ok_row:
#             try:
#                 if arr[1] is not None:
#                     last_nums_annonce.append(arr[1])
#             except:
#                 pass

In [31]:
# last_nums_annonce

In [82]:
# last_nums_other = []
# for index, row in df[df.y == 'Остальное'].iterrows():
#     ok_row = row['form_feat']
#     if ok_row is not None:
#         for arr in ok_row:
#             try:
#                 if arr[1] is not None:
#                     last_nums_other.append(arr[1])
#             except:
#                 pass

In [33]:
# last_nums_other

In [83]:
# np.mean(last_nums_annonce), np.median(last_nums_annonce), np.max(last_nums_annonce), np.min(last_nums_annonce)

In [84]:
# np.mean(last_nums_other), np.median(last_nums_other), np.max(last_nums_other), np.min(last_nums_other)

In [75]:
# mean, median, max по данному тексту

## Добавляем новые признаки

In [94]:
# new_features = []

# for row in df.form_feat:
#     try:
#         nums = []
#         for item in row:
#             nums.append(item[0])
#         new_features.append([np.mean(nums), np.median(nums), np.max(nums)])
#     except:
#         new_features.append(None)
        

In [127]:
# df['new_features'] = new_features

In [23]:
new_df = df.dropna()

In [24]:
# df.head()

In [25]:
new_df.y.value_counts()

Остальное    14068
Анонс          164
Name: y, dtype: int64

In [26]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [27]:
X = new_df[['X', 'n_of_dates_fut', 'n_of_intervals']]
y = new_df['y']

In [28]:
count_vect = CountVectorizer()
X_2 = count_vect.fit_transform(X.X)
tf_transformer = TfidfTransformer()
X_2 = tf_transformer.fit_transform(X_2)

In [29]:
# X_test_Xs = count_vect.transform(X_test.X)
# X_test_Xs = tf_transformer.transform(X_test_Xs)

In [30]:
kbest = SelectKBest(chi2, k=1000)
kbest.fit(X_2, y)
X_selected = kbest.transform(X_2)

In [31]:
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
# list(X_train.new_features)

In [32]:
len(list(X.n_of_dates_fut))

14232

In [33]:
type(X_selected.todense())

numpy.matrixlib.defmatrix.matrix

In [34]:
X_selected.todense().shape, len(X.n_of_dates_fut)

((14232, 1000), 14232)

In [35]:
# X.n_of_dates_fut

In [36]:
# добавим наши формальные признаки
X_prefinal = np.append(X_selected.todense(), list(X.n_of_dates_fut), axis=1)

In [37]:
X_final = np.append(X_prefinal, list(X.n_of_intervals), axis=1)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, stratify=y)

In [39]:
# from sklearn.decomposition import PCA

In [40]:
# pca = PCA(n_components=20)

In [41]:
# X_pca_tr= pca.fit(X_train_final)

In [42]:
model = XGBClassifier(max_depth=3, n_estimators=1500)

In [43]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [44]:
y_pred = model.predict(X_test)

In [45]:
print(classification_report(y_test, list(y_pred)))

             precision    recall  f1-score   support

      Анонс       0.94      0.65      0.77        49
  Остальное       1.00      1.00      1.00      4221

avg / total       1.00      1.00      1.00      4270



### Результаты на 20 тысячах случайных текстов

In [46]:
def get_features():
    n_dates_fut_array = []
    n_dates_interval = []
    path1 = '/Users/Valeriya/Desktop/Interfax/RandomDocsfeatures/'
#     path2 = '/Users/Valeriya/Desktop/Interfax/features_all_22november/'
#     curr_path = ''
    for i in tqdm.tqdm_notebook(df_20k.filename):
        if i.endswith('html'):
            fromdf = i.strip('.html')
            fromdf = re.sub('[0-9]+_', '', fromdf)
        elif i.endswith('xml'):
            fromdf = i.strip('.xml')
            fromdf = re.sub('[0-9]+_', '', fromdf)

        name = 'dateFeature_'+fromdf+'.txt'

#         if os.path.exists(path2 + name):
#             curr_path = path2
        if os.path.exists(path1+name):
            curr_path = path1
        else:
            n_dates_fut_array.append(None)
            n_dates_interval.append(None)
            continue

    #         print('ok')
        f = open(curr_path+name)
        file = f.readlines()
    #     feature_vec_for_row = []
        n_dates_fut = 0
        n_dates_2weeks = 0
        for row in file:

            nums = row.split('|')[0].split('\t')
            if len(nums) == 2:
                num_fut = nums[0].strip('\ufeff').strip()
                num_interval = nums[1].strip('\ufeff').strip()
                n_dates_fut +=1
                if int(num_interval) >= 14:
                    n_dates_2weeks += 1
            elif len(nums) == 1:
                pass

        n_dates_fut_array.append([n_dates_fut])
        n_dates_interval.append([n_dates_2weeks])
        f.close()
        
    return n_dates_fut_array, n_dates_interval

In [51]:
# n = 0
def read_txt_to_predict(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            f = open(open_name)
            try:
                cleantext = BeautifulSoup(f, 'lxml').text
                yield cleantext, filename
#                 n += 1
#                 yield open_name
                f.close()
            except:
                f.close()
                pass

In [52]:
### collect docs

In [53]:
data_to_predict = []

In [54]:
for i in tqdm.tqdm_notebook(read_txt_to_predict('/Users/Valeriya/Desktop/Interfax/RandomDocs')):
        data_to_predict.append(i)

In [55]:
len(data_to_predict)

19996

In [56]:
df_20k = pd.DataFrame(data_to_predict, columns=['X', 'filename'])

In [57]:
dates_future, dates_intervals = get_features()

In [58]:
df_20k['n_of_dates_fut'] = dates_future
df_20k['n_of_intervals'] = dates_intervals

In [59]:
new_df_20 = df_20k.dropna()

In [64]:
file = open('results_26Jan_random_docs.txt', 'a')

In [67]:
# for txt, dates, intervals in zip(new_df_20.X, new_df_20.n_of_dates_fut, new_df_20.n_of_intervals):
#     X_2 = count_vect.transform([txt])
#     X_2 = tf_transformer.transform(X_2)
#     X20_selected= kbest.transform(X_2)
#     X_prefinal = np.append(X20_selected.todense(), dates, axis=1)
#     X_final = np.append(X_prefinal, intervals, axis=1)
#     print(X_final)
#     break

In [76]:
# count_vect = CountVectorizer()
X_2 = count_vect.transform(new_df_20.X)
# tf_transformer = TfidfTransformer()
X_2 = tf_transformer.transform(X_2)

In [77]:
X20_selected= kbest.transform(X_2)

In [78]:
X_prefinal = np.append(X20_selected.todense(), list(new_df_20.n_of_dates_fut), axis=1)

In [79]:
X_final = np.append(X_prefinal, list(new_df_20.n_of_intervals), axis=1)

In [80]:
y_predict_20k_new_features = model.predict(X_final)

In [87]:
with open('results_26Jan_random_docs.txt', 'a') as file:
    for prediction, filename in zip(y_predict_20k_new_features, new_df_20.filename):
        file.write(prediction + ' ')
        file.write(filename)
        file.write('\n')